# Compiling the different versions of Micro-Aevol 2

In [1]:
import os
import subprocess

In [2]:
# parameters

experiment_params = {
    "dna_sizes": [500, 5000, 50000], # size of DNA
    "mutation_rates": [0.0001, 0.00001, 0.000001], # mutation rate
    "world_sizes": [32, 64, 128, 256, 1024], # size of the world
}

# get project names from dirs in ./aevols directory
project_names = [d for d in os.listdir("./aevols") if os.path.isdir(os.path.join("./aevols", d))]
print(project_names)

['aevol_0']


## Compiling & Run executables

In [10]:
KOKKOS_LIB_CMAKE = "Kokkos_DIR=/home/onyr/Kokkos/kokkos_install/lib/cmake/Kokkos"

for project_name in project_names:
    # create directory build
    build_dir = "./aevols/" + project_name + "/build"
    if not os.path.exists(build_dir):
        os.makedirs(build_dir)
    else:
        print("build directory already exists")
        print("removing previous build directory")
        subprocess.run(["rm", "-rf", build_dir])
    
    # run cmake to generate makefiles
    subprocess.run(["cmake", "-B", f"./aevols/{project_name}/build", "-S", f"./aevols/{project_name}", "-D", KOKKOS_LIB_CMAKE])

    # # run make to compile
    # subprocess.run(["make", "-C", f"./aevols/{project_name}/build"])

    # for dna_size in experiment_params["dna_sizes"]:
    #     for mutation_rate in experiment_params["mutation_rates"]:
    #         for world_size in experiment_params["world_sizes"]:
    #             print("Running {} (exec_type: {}) with dna_size={}, mutation_rate={}, world_size={}".format(project_name, exec_type, dna_size, mutation_rate, world_size))
                
    #             # run the executable
    #             executable_name = "micro_aevol_cpu" if (exec_type == "nameCpu") else "micro_aevol_cuda"
    #             params = f" -w {world_size} -h {world_size} -g {dna_size} -m {mutation_rate}"
    #             subprocess.run(["./aevols/" + project_name + "/build/" + executable_name, params])

build directory already exists
removing previous build directory
-- The C compiler identification is GNU 11.3.0
-- The CXX compiler identification is GNU 11.3.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring incomplete, errors occurred!
See also "/home/onyr/Documents/code/OT5_micro-aevol2/src/aevols/aevol_0/build/CMakeFiles/CMakeOutput.log".


CMake Error at CMakeLists.txt:21 (find_package):
  By not providing "FindKokkos.cmake" in CMAKE_MODULE_PATH this project has
  asked CMake to find a package configuration file provided by "Kokkos", but
  CMake did not find one.

  Could not find a package configuration file provided by "Kokkos" with any
  of the following names:

    KokkosConfig.cmake
    kokkos-config.cmake

  Add the installation prefix of "Kokkos" to CMAKE_PREFIX_PATH or set
  "Kokkos_DIR" to a directory containing one of the above files.  If "Kokkos"
  provides a separate development package or SDK, be sure it has been
  installed.


